# MP305 Network Flow Models II - Michael Tuite

In [1]:
from IPython.display import display, Math, Latex

## Overview 

This file contains a number of Python functions for finding the maximal flow through a network $G$ subject to minimal cost using the Ford Fulkerson Algorithm.

The network graph $G$ is stored in a set `G` of two element tuples `(i,j)` describing the directed arcs $(i,j)$ of $G$.

It is assumed that node number $1$ is the source and the greatest node `Nsink` is the sink.  
Thus `G={(1,2),(2,3),(1,3)}`  describes a network with 3 nodes where node 1 is the source and node 3 is the sink.

The capacity $c(i,j)$, flow $\phi(i,j)$ and cost $l(i,j)$ of the arc $(i,j)$ of $G$ are stored in `c[i][j]`, `phi[i][j]` and `l[i][j]`. 
Here `c`, `phi` and `l` are Python lists.

## Python Functions
### The `Initialise(G)` function
Having defined the network $G$, initialise `c`, `phi` and `l` values to zero via the `Initialise` function before defining their values in any particular example.  The global variable `Nsink`,  the sink node of $G$, is also found by the `Initialise` function .

### The main `Iterate(G)`function 
This implements the full algorithm to find the maximum flow with minimal cost. 

## The `Iterate(G)` function is based on a number of other Python functions:

### `Flows(G)` 
This checks for conservation of flow and prints out all of the current flows for G and the total cost of this flow. 

###  `Links(G)`
This finds all arcs `(i,j1)`, ` (i,j2)`,  ... *out* of node `i` of `G`.  The nodes `j1,j2,..` are stored in a global list of sets `Out`.

###  `SourceSink(G)`
This finds all of the paths from source to sink in any network `G` and results in a global set `SinkPaths` of such paths.

###  `IncremNet(G)`
This finds the Incremental Network `Gp` associated with the current flow of the network `G`.

###  `Newflows(G)`
This updates the flows `phi` of `G` according to the best chain found through `Gp`. If the maximal flow is reached, then this is indicated and the maximum flow value is outputed. Otherwise, the output is: the change in flow (`eps`), the cost of the best chain, and the best chain.

###  `Iterate(G)`
Implements the full algorithm to find the maximum flow with minimal cost. 
The output is as follows:

(1) The incremental network `Gp`.

(2) The paths through `Gp` from source to sink. 

(3) The output of `Newflows(G)`. 

(4) The output of `Flows(G)` giving the current flows and cost of `G`.


In [2]:
def Initialise (Gin):
    global c,phi,l,cp,lp,Nsink
    Nsink=1
    for arc in Gin:
        i,j=arc
        Nsink=max(Nsink,i,j)
    # for convenience c[i][j] is capacity of arc [i,j]
    c=[[0 for i in range(Nsink+1)] for j in range(Nsink+1)] 
    phi=[[0 for i in range(Nsink+1)] for j in range(Nsink+1)]
    l=[[0 for i in range(Nsink+1)] for j in range(Nsink+1)]
    cp=[[0 for i in range(Nsink+1)] for j in range(Nsink+1)]
    lp=[[0 for i in range(Nsink+1)] for j in range(Nsink+1)]
    print("All values of c,phi and l initialised to zero")


In [3]:
def Flows (Gin):
    global Nsink,l,phi
    Flowin=[0 for i in range(Nsink+1)]
    Flowout=[0 for i in range(Nsink+1)]
    for arc in Gin:
        i,j=arc
        Flowin[j] = Flowin[j] + phi[i][j]
        Flowout[i] = Flowout[i] + phi[i][j]
    for k in range(2,Nsink):
        if Flowin[k] != Flowout[k]:
            print("*** ERROR *** Flow not conserved at node", k)
    if Flowout[1] != Flowin[Nsink]:
        print("*** ERROR *** Flow not conserved at source or sink")
    Totalcost = 0
    for arc in Gin:
        i,j=arc
        phi_ij = phi[i][j]
        Totalcost = Totalcost + l[i][j]*phi_ij
        print(arc," has flow ",phi_ij)
    print("Total Cost is ", Totalcost)


In [4]:
def Links (Gin):
    global Nsink,Out
    Out=[set() for k in range(Nsink)] # labelled 0..Nsink-1
    for arc in Gin:
        i,j = arc
        Out[i - 1] = Out[i - 1] | set([j])

In [5]:
def SourceSink(Gin):
# finds all paths SinkPaths from source 1 to sink Nsink of network G 
    global Nsink,SinkPaths
    Links(Gin)
    Paths = set() # current paths from source stored as set of tuples
    SinkPaths = set() # paths from source to sink Nsink stored as set of tuples
    path = 1 # source node label
    for node in Out[0]:# need out edge from node 1
        pathn = (path,node)
        if node == Nsink:
            SinkPaths = SinkPaths | set([pathn])
        else:
            Paths = Paths | set([pathn])
    Npaths = len(Paths)
    while (0 < Npaths):
        NewPaths = set()
        for oldpath in Paths:
            nold = len(oldpath)
            m = oldpath[-1] # last node in tuple oldpath
            for mout in Out[m-1]:
                if not mout in oldpath:
                    if mout == Nsink:
                        SinkPaths = SinkPaths | set([oldpath+tuple([Nsink])])
                    else:
                        NewPaths = NewPaths | set([oldpath+tuple([mout])])
        Paths = NewPaths
        Npaths = len(Paths)
    print("Paths from source to sink: ",SinkPaths)

In [6]:
def Newflows(Gin):
# A procedure to modify original flows on Gin along SinkPaths of Gp with minimal cost
    global Gp,phi,c,l,cp,lp,ArcSign,Out
    SourceSink(Gp)
    if SinkPaths == set():
        Links(Gin)
        Flow = 0
        for node in Out[0]:
            Flow = Flow + phi[1][node]
        Cost=0
        for arc in Gin:
            i,j=arc
            Cost=Cost+l[i][j]*phi[i][j]
        print("Maximal flow found:", Flow, " with minimal cost ", Cost)
    else:
        for k in range(len(SinkPaths)):
            cost = 0
            epset = set()
            path=list(SinkPaths)[k]
            for n in range(0, len(path)-1):
                i = path[n]; j = path[n+1];  epset = epset | set([cp[i][j]]); cost = lp[i][j] + cost
            eps = min(tuple(epset))
            if k == 0: # first path
                mincost = cost; bestpath = path; besteps = eps
            elif cost < mincost: 
                mincost = cost; bestpath = path; besteps = eps
        print("A best path in Gp is ", bestpath, " of minimum cost ", mincost)
        print("The min capacity on this path is epsilon ", besteps)
        print("The min cost is ", mincost)
        for k in range(0, len(bestpath) - 1):
            i = bestpath[k]; j = bestpath[k+1]
            if ArcSign[i][j] == 1:
                phinewij = phi[i][j] + besteps; phi[i][j]=phinewij
            else:
                phinewji=phi[j][i] = phi[j][i] - besteps; phi[j][i]=phinewji
    #print("Flow=",Flow)

In [7]:
def Iterate(Gin):
    IncremNet(Gin)
    Newflows(Gin)
    for arc in Gin:
        i,j=arc
        print((i,j)," flow = ", phi[i][j])

In [8]:
def IncremNet(Gin):
# procedure to create incremental network Gp from given flow network G 
    global Gp,Nsink,phi,c,l,cp,lp,ArcSign
# define lists for ArcSign, cp and lp  (indexed by 0..Nsink-1)
    cp=[[0 for i in range(Nsink+1)] for j in range(Nsink+1)]
    lp=[[0 for i in range(Nsink+1)] for j in range(Nsink+1)]
    ArcSign=[[0 for i in range(Nsink+1)] for j in range(Nsink+1)]
    Gp=set([])
    for arc in Gin:
        i,j=arc
        pij = phi[i][j]; pji = phi[j][i]; cij = c[i][j]; lij = l[i][j]
        if (pij < cij and (pji == 0 or not (j,i) in Gin)): # ij arc
            #Gp edges, capacitites and costs added
            cpij = cij - pij; cp[i][j] = cpij; lpij = lij; lp[i][j] =lpij
            ArcSign[i][j] = 1
            Gp=Gp | {(i,j)}
        if pij>0: # ji arc
            cpji = pij; cp[j][i] = cpji; lpji=-lij; lp[j][i] = lpji
            ArcSign[j][i] = -1
            Gp=Gp | {(j,i)}
    print("Incremental Network:",Gp)

# Q 1. Power Station/Coal Field Supply
### Find the maximal flow for minimal cost for the network below where (capacity,cost) is shown:

![Network](Lab2_1.jpg)


## This is the Coalfield/Power station supply problem discussed in the notes
### * Find the incremental network, its capacities and costs at each iteration.

In [9]:
G={(1,2),(1,3),(2,4),(3,5),
   (2,5),(3,4),(4,6),(5,6)}

In [10]:
len(G)

8

In [11]:
Initialise(G)

All values of c,phi and l initialised to zero


## Input capacities

In [12]:
c[1][2]=5; c[1][3]=3; c[2][4]=5; c[3][5]=3 
c[2][5]=5; c[3][4]=3; c[4][6]=4; c[5][6]=4

## Input costs

In [13]:
l[1][2]=2; l[1][3]=3; l[2][4]=3; l[3][5]=6 
l[2][5]=2; l[3][4]=2; l[4][6]=0; l[5][6]=0

In [14]:
Flows(G)

(2, 4)  has flow  0
(1, 2)  has flow  0
(3, 4)  has flow  0
(4, 6)  has flow  0
(5, 6)  has flow  0
(2, 5)  has flow  0
(1, 3)  has flow  0
(3, 5)  has flow  0
Total Cost is  0


In [15]:
Iterate(G)

Incremental Network: {(2, 4), (1, 2), (2, 5), (3, 4), (5, 6), (4, 6), (1, 3), (3, 5)}
Paths from source to sink:  {(1, 2, 5, 6), (1, 3, 4, 6), (1, 2, 4, 6), (1, 3, 5, 6)}
A best path in Gp is  (1, 2, 5, 6)  of minimum cost  4
The min capacity on this path is epsilon  4
The min cost is  4
(2, 4)  flow =  0
(1, 2)  flow =  4
(3, 4)  flow =  0
(4, 6)  flow =  0
(5, 6)  flow =  4
(2, 5)  flow =  4
(1, 3)  flow =  0
(3, 5)  flow =  0


### Continue to iterate until max flow for minimal cost found

In [16]:
Iterate(G)

Incremental Network: {(2, 4), (1, 2), (2, 1), (3, 4), (6, 5), (4, 6), (2, 5), (1, 3), (3, 5), (5, 2)}
Paths from source to sink:  {(1, 3, 5, 2, 4, 6), (1, 3, 4, 6), (1, 2, 4, 6)}
A best path in Gp is  (1, 3, 4, 6)  of minimum cost  5
The min capacity on this path is epsilon  3
The min cost is  5
(2, 4)  flow =  0
(1, 2)  flow =  4
(3, 4)  flow =  3
(4, 6)  flow =  3
(5, 6)  flow =  4
(2, 5)  flow =  4
(1, 3)  flow =  3
(3, 5)  flow =  0


In [17]:
Iterate(G)

Incremental Network: {(2, 4), (1, 2), (2, 1), (6, 5), (4, 3), (3, 1), (4, 6), (6, 4), (2, 5), (3, 5), (5, 2)}
Paths from source to sink:  {(1, 2, 4, 6)}
A best path in Gp is  (1, 2, 4, 6)  of minimum cost  5
The min capacity on this path is epsilon  1
The min cost is  5
(2, 4)  flow =  1
(1, 2)  flow =  5
(3, 4)  flow =  3
(4, 6)  flow =  4
(5, 6)  flow =  4
(2, 5)  flow =  4
(1, 3)  flow =  3
(3, 5)  flow =  0


In [18]:
Iterate(G)

Incremental Network: {(2, 4), (2, 1), (6, 5), (4, 3), (3, 1), (6, 4), (4, 2), (2, 5), (3, 5), (5, 2)}
Paths from source to sink:  set()
Maximal flow found: 8  with minimal cost  36
(2, 4)  flow =  1
(1, 2)  flow =  5
(3, 4)  flow =  3
(4, 6)  flow =  4
(5, 6)  flow =  4
(2, 5)  flow =  4
(1, 3)  flow =  3
(3, 5)  flow =  0


# Q2. (*)  
## A road network is shown below with the capacity and time taken per car on each road indicated.
![Network](Lab2_2.jpg)


## * Find the maximal flow through the network for minimal total travel time for all cars from A to B. 
## * Compare this to the flow from B to A.

### Note: You can use the network and capacitites you created in the first lab Network Flows I, Question 3.

In [18]:
G2 = {(1, 2),(2, 1),(1, 3),(3, 1),(3, 4),(4, 3),(4, 2),(2, 5),(4, 6),
      (6, 4),(3, 7),(7, 6),(5, 6),(5, 10),(10, 5),(7, 10),(10, 7)}
len(G2)

17

In [19]:
Initialise(G2)

All values of c,phi and l initialised to zero


In [20]:
c[1][2] = 5; c[2][1] = 5; c[1][3] = 4; c[3][1] = 4;
c[3][4] = 5; c[4][3] = 5; c[4][2] = 4; c[2][5] = 3;
c[4][6] = 5; c[6][4] = 5; c[3][7] = 3; c[7][6] = 4;
c[5][6] = 2; c[5][10] = 6; c[10][5] = 6; c[7][10] = 4;
c[10][7] = 4

In [21]:
l[1][2] = 3; l[2][1] = 3; l[1][3] = 2; l[3][1] = 2;
l[3][4] = 1; l[4][3] = 1; l[4][2] = 3; l[2][5] = 2;
l[4][6] = 2; l[6][4] = 2; l[3][7] = 2; l[7][6] = 3;
l[5][6] = 3; l[5][10] = 3; l[10][5] = 3; l[7][10] = 1;
l[10][7] = 1

In [22]:
Flows(G2)

(10, 5)  has flow  0
(1, 2)  has flow  0
(2, 1)  has flow  0
(3, 4)  has flow  0
(4, 3)  has flow  0
(3, 1)  has flow  0
(3, 7)  has flow  0
(7, 10)  has flow  0
(4, 6)  has flow  0
(6, 4)  has flow  0
(4, 2)  has flow  0
(10, 7)  has flow  0
(5, 10)  has flow  0
(7, 6)  has flow  0
(5, 6)  has flow  0
(2, 5)  has flow  0
(1, 3)  has flow  0
Total Cost is  0


In [23]:
Iterate(G2)

Incremental Network: {(10, 5), (1, 2), (2, 1), (3, 4), (4, 3), (3, 1), (3, 7), (4, 6), (6, 4), (5, 10), (7, 6), (2, 5), (1, 3), (7, 10), (10, 7), (4, 2), (5, 6)}
Paths from source to sink:  {(1, 2, 5, 6, 4, 3, 7, 10), (1, 3, 7, 6, 4, 2, 5, 10), (1, 2, 5, 10), (1, 3, 7, 10), (1, 3, 4, 2, 5, 10)}
A best path in Gp is  (1, 3, 7, 10)  of minimum cost  5
The min capacity on this path is epsilon  3
The min cost is  5
(10, 5)  flow =  0
(1, 2)  flow =  0
(2, 1)  flow =  0
(3, 4)  flow =  0
(4, 3)  flow =  0
(3, 1)  flow =  0
(3, 7)  flow =  3
(7, 10)  flow =  3
(4, 6)  flow =  0
(6, 4)  flow =  0
(4, 2)  flow =  0
(10, 7)  flow =  0
(5, 10)  flow =  0
(7, 6)  flow =  0
(5, 6)  flow =  0
(2, 5)  flow =  0
(1, 3)  flow =  3


In [24]:
Iterate(G2)

Incremental Network: {(10, 5), (1, 2), (2, 1), (3, 4), (4, 3), (3, 1), (4, 6), (6, 4), (7, 3), (5, 10), (7, 6), (2, 5), (1, 3), (7, 10), (10, 7), (4, 2), (5, 6)}
Paths from source to sink:  {(1, 3, 4, 2, 5, 10), (1, 2, 5, 10)}
A best path in Gp is  (1, 2, 5, 10)  of minimum cost  8
The min capacity on this path is epsilon  3
The min cost is  8
(10, 5)  flow =  0
(1, 2)  flow =  3
(2, 1)  flow =  0
(3, 4)  flow =  0
(4, 3)  flow =  0
(3, 1)  flow =  0
(3, 7)  flow =  3
(7, 10)  flow =  3
(4, 6)  flow =  0
(6, 4)  flow =  0
(4, 2)  flow =  0
(10, 7)  flow =  0
(5, 10)  flow =  3
(7, 6)  flow =  0
(5, 6)  flow =  0
(2, 5)  flow =  3
(1, 3)  flow =  3


In [25]:
Iterate(G2)

Incremental Network: {(10, 5), (1, 2), (2, 1), (3, 4), (4, 3), (3, 1), (4, 6), (6, 4), (7, 3), (5, 10), (7, 6), (1, 3), (5, 2), (7, 10), (10, 7), (4, 2), (5, 6)}
Paths from source to sink:  set()
Maximal flow found: 6  with minimal cost  39
(10, 5)  flow =  0
(1, 2)  flow =  3
(2, 1)  flow =  0
(3, 4)  flow =  0
(4, 3)  flow =  0
(3, 1)  flow =  0
(3, 7)  flow =  3
(7, 10)  flow =  3
(4, 6)  flow =  0
(6, 4)  flow =  0
(4, 2)  flow =  0
(10, 7)  flow =  0
(5, 10)  flow =  3
(7, 6)  flow =  0
(5, 6)  flow =  0
(2, 5)  flow =  3
(1, 3)  flow =  3


# FROM B TO A

In [26]:
G3 = {(10, 2),(2, 10),(10, 3),(3, 10),(3, 4),(4, 3),(4, 2),(2, 5),(4, 6),(6, 4),(3, 7),(7, 6),(5, 6),(5, 1),(1, 5),(7, 1),(1, 7)}
len(G2)

17

In [27]:
Initialise(G3)

All values of c,phi and l initialised to zero


In [28]:
c[10][2] = 5; c[2][10] = 5; c[10][3] = 4; c[3][10] = 4;
c[3][4] = 5; c[4][3] = 5; c[4][2] = 4; c[2][5] = 3;
c[4][6] = 5; c[6][4] = 5; c[3][7] = 3; c[7][6] = 4;
c[5][6] = 2; c[5][1] = 6; c[1][5] = 6; c[7][1] = 4;
c[1][7] = 4

In [29]:
l[10][2] = 3; l[2][10] = 3; l[10][3] = 2; l[3][10] = 2;
l[3][4] = 1; l[4][3] = 1; l[4][2] = 3; l[2][5] = 2;
l[4][6] = 2; l[6][4] = 2; l[3][7] = 2; l[7][6] = 3;
l[5][6] = 3; l[5][1] = 3; l[1][5] = 3; l[7][1] = 1;
l[1][7] = 1

In [30]:
Flows(G3)

(7, 1)  has flow  0
(3, 4)  has flow  0
(4, 3)  has flow  0
(1, 5)  has flow  0
(3, 7)  has flow  0
(2, 10)  has flow  0
(4, 6)  has flow  0
(3, 10)  has flow  0
(4, 2)  has flow  0
(6, 4)  has flow  0
(5, 1)  has flow  0
(1, 7)  has flow  0
(7, 6)  has flow  0
(5, 6)  has flow  0
(10, 3)  has flow  0
(2, 5)  has flow  0
(10, 2)  has flow  0
Total Cost is  0


In [31]:
Iterate(G3)

Incremental Network: {(3, 4), (4, 3), (1, 5), (3, 7), (2, 10), (4, 6), (3, 10), (6, 4), (5, 1), (7, 6), (10, 3), (2, 5), (7, 1), (4, 2), (1, 7), (5, 6), (10, 2)}
Paths from source to sink:  {(1, 7, 6, 4, 2, 10), (1, 5, 6, 4, 2, 10), (1, 7, 6, 4, 3, 10), (1, 5, 6, 4, 3, 10)}
A best path in Gp is  (1, 7, 6, 4, 3, 10)  of minimum cost  9
The min capacity on this path is epsilon  4
The min cost is  9
(7, 1)  flow =  0
(3, 4)  flow =  0
(4, 3)  flow =  4
(1, 5)  flow =  0
(3, 7)  flow =  0
(2, 10)  flow =  0
(4, 6)  flow =  0
(3, 10)  flow =  4
(4, 2)  flow =  0
(6, 4)  flow =  4
(5, 1)  flow =  0
(1, 7)  flow =  4
(7, 6)  flow =  4
(5, 6)  flow =  0
(10, 3)  flow =  0
(2, 5)  flow =  0
(10, 2)  flow =  0


In [32]:
Iterate(G3)

Incremental Network: {(7, 1), (3, 4), (4, 3), (1, 5), (3, 7), (2, 10), (4, 6), (6, 4), (4, 2), (5, 1), (6, 7), (5, 6), (10, 3), (2, 5), (10, 2)}
Paths from source to sink:  {(1, 5, 6, 4, 2, 10)}
A best path in Gp is  (1, 5, 6, 4, 2, 10)  of minimum cost  14
The min capacity on this path is epsilon  1
The min cost is  14
(7, 1)  flow =  0
(3, 4)  flow =  0
(4, 3)  flow =  4
(1, 5)  flow =  1
(3, 7)  flow =  0
(2, 10)  flow =  1
(4, 6)  flow =  0
(3, 10)  flow =  4
(4, 2)  flow =  1
(6, 4)  flow =  5
(5, 1)  flow =  0
(1, 7)  flow =  4
(7, 6)  flow =  4
(5, 6)  flow =  1
(10, 3)  flow =  0
(2, 5)  flow =  0
(10, 2)  flow =  0


In [33]:
Iterate(G3)

Incremental Network: {(2, 4), (7, 1), (3, 4), (6, 5), (4, 3), (1, 5), (3, 7), (2, 10), (4, 6), (5, 1), (4, 2), (6, 7), (5, 6), (10, 3), (2, 5), (10, 2)}
Paths from source to sink:  set()
Maximal flow found: 5  with minimal cost  50
(7, 1)  flow =  0
(3, 4)  flow =  0
(4, 3)  flow =  4
(1, 5)  flow =  1
(3, 7)  flow =  0
(2, 10)  flow =  1
(4, 6)  flow =  0
(3, 10)  flow =  4
(4, 2)  flow =  1
(6, 4)  flow =  5
(5, 1)  flow =  0
(1, 7)  flow =  4
(7, 6)  flow =  4
(5, 6)  flow =  1
(10, 3)  flow =  0
(2, 5)  flow =  0
(10, 2)  flow =  0


 # Q.3 (*) Soft Drinks Stock Control
 ## A soft drinks firm buys fruit at the beginning of each month $i$ at a cost per 100kg $p_i$ in units of 1000 Euro. 
 ## The firm can store up 2000kg of fruit at any given time where cost in units 1000 Euro of refrigeration per month per 100kg is $r_i$. 
## The consumption requirements are $c_i$ per month in 100kg units. 

## Based on last year's figures,  the following estimates have been made:

In [34]:
Latex(r"""
\begin{array}{|l|l|l|l|l|l|l|l|l|l|l|l|l|}
\hline
i & Jan & Feb & Mar & April & May & June & July & Aug & Sept & Oct & Nov & 
Dec \\ \hline
p_{i} & 18 & 17 & 17 & 15 & 12 & 8 & 7 & 6 & 9 & 12 & 14 & 17 \\ \hline
r_{i} & 1 & 1 & 2 & 2 & 3 & 5 & 6 & 6 & 5 & 3 & 2 & 1 \\ \hline
c_{i} & 9 & 6 & 6 & 7 & 11 & 14 & 16 & 18 & 15 & 10 & 7 & 6 \\ \hline
\end{array}
""")

<IPython.core.display.Latex object>

## (a) Find the best purchasing schedule starting from January based on these estimates assuming that the firm has no fruit in storage on Jan 1st. 
## How much will fruit cost for the year and how much will be spent on refrigeration?

## (b) Suppose that the firm has 500kg of fruit in storage at the beginning of the year.
## What is the best purchasing schedule that ensures that 500kg of fruit is again in storage at the very end of the year?

In [35]:
def Initialise20 (Gin):
    global c,phi,l,cp,lp,Nsink
    Nsink=1
    for arc in Gin:
        i,j=arc
        Nsink=max(Nsink,i,j)
    # for convenience c[i][j] is capacity of arc [i,j]
    c=[[20 for i in range(Nsink+1)] for j in range(Nsink+1)] 
    phi=[[0 for i in range(Nsink+1)] for j in range(Nsink+1)]
    l=[[0 for i in range(Nsink+1)] for j in range(Nsink+1)]
    cp=[[0 for i in range(Nsink+1)] for j in range(Nsink+1)]
    lp=[[0 for i in range(Nsink+1)] for j in range(Nsink+1)]
    print("All values of phi and l initialised to zero and c to 20")


In [36]:
G4 = {(1,10), (10,11),
     (1,11), (1,12), (1,13),
     (11,21), (21,12), (12,22), (22,13), (13,23),
     (21,30), (22,30), (23,30)} 
len(G4)

13

In [37]:
Initialise20(G4)

All values of phi and l initialised to zero and c to 20


In [38]:
c[1][10] = 4; c[10][11] = 4; 
c[1][11] = 5; c[1][12] = 5; c[1][13] = 5;
c[11][21] = 5; c[21][12] = 5; c[12][22] = 5;
c[22][13] = 5; c[13][23] = 5; 
c[21][30] = 3; c[22][30] = 5; c[23][30] = 6

In [39]:
l[1][10] = 0; l[10][11] = 0; 
l[1][11] = 3; l[1][12] = 2; l[1][13] = 4;
l[11][21] = 0; l[21][12] = 0; l[12][22] = 0;
l[22][13] = 0; l[13][23] = 0; 
l[21][30] = 0; l[22][30] = 0; l[23][30] = 0

In [40]:
Flows(G4)

(1, 12)  has flow  0
(10, 11)  has flow  0
(13, 23)  has flow  0
(1, 11)  has flow  0
(22, 30)  has flow  0
(21, 12)  has flow  0
(12, 22)  has flow  0
(22, 13)  has flow  0
(1, 13)  has flow  0
(11, 21)  has flow  0
(1, 10)  has flow  0
(21, 30)  has flow  0
(23, 30)  has flow  0
Total Cost is  0


In [41]:
Iterate(G4)

Incremental Network: {(1, 12), (10, 11), (13, 23), (1, 11), (22, 30), (21, 12), (12, 22), (22, 13), (1, 13), (11, 21), (1, 10), (21, 30), (23, 30)}
Paths from source to sink:  {(1, 11, 21, 30), (1, 10, 11, 21, 30), (1, 11, 21, 12, 22, 30), (1, 10, 11, 21, 12, 22, 13, 23, 30), (1, 13, 23, 30), (1, 11, 21, 12, 22, 13, 23, 30), (1, 10, 11, 21, 12, 22, 30), (1, 12, 22, 13, 23, 30), (1, 12, 22, 30)}
A best path in Gp is  (1, 10, 11, 21, 30)  of minimum cost  0
The min capacity on this path is epsilon  3
The min cost is  0
(1, 12)  flow =  0
(10, 11)  flow =  3
(13, 23)  flow =  0
(1, 11)  flow =  0
(22, 30)  flow =  0
(21, 12)  flow =  0
(12, 22)  flow =  0
(22, 13)  flow =  0
(1, 13)  flow =  0
(11, 21)  flow =  3
(1, 10)  flow =  3
(21, 30)  flow =  3
(23, 30)  flow =  0


In [42]:
Iterate(G4)

Incremental Network: {(1, 12), (10, 11), (11, 10), (13, 23), (1, 11), (22, 30), (30, 21), (10, 1), (21, 12), (12, 22), (22, 13), (1, 13), (11, 21), (1, 10), (21, 11), (23, 30)}
Paths from source to sink:  {(1, 13, 23, 30), (1, 11, 21, 12, 22, 30), (1, 10, 11, 21, 12, 22, 13, 23, 30), (1, 11, 21, 12, 22, 13, 23, 30), (1, 10, 11, 21, 12, 22, 30), (1, 12, 22, 13, 23, 30), (1, 12, 22, 30)}
A best path in Gp is  (1, 10, 11, 21, 12, 22, 13, 23, 30)  of minimum cost  0
The min capacity on this path is epsilon  1
The min cost is  0
(1, 12)  flow =  0
(10, 11)  flow =  4
(13, 23)  flow =  1
(1, 11)  flow =  0
(22, 30)  flow =  0
(21, 12)  flow =  1
(12, 22)  flow =  1
(22, 13)  flow =  1
(1, 13)  flow =  0
(11, 21)  flow =  4
(1, 10)  flow =  4
(21, 30)  flow =  3
(23, 30)  flow =  1


In [43]:
Iterate(G4)

Incremental Network: {(11, 10), (30, 23), (1, 11), (23, 13), (12, 22), (13, 22), (21, 11), (30, 21), (22, 12), (1, 12), (13, 23), (22, 30), (10, 1), (21, 12), (22, 13), (1, 13), (11, 21), (12, 21), (23, 30)}
Paths from source to sink:  {(1, 13, 23, 30), (1, 13, 22, 30), (1, 11, 21, 12, 22, 30), (1, 11, 21, 12, 22, 13, 23, 30), (1, 12, 22, 13, 23, 30), (1, 12, 22, 30)}
A best path in Gp is  (1, 12, 22, 13, 23, 30)  of minimum cost  2
The min capacity on this path is epsilon  4
The min cost is  2
(1, 12)  flow =  4
(10, 11)  flow =  4
(13, 23)  flow =  5
(1, 11)  flow =  0
(22, 30)  flow =  0
(21, 12)  flow =  1
(12, 22)  flow =  5
(22, 13)  flow =  5
(1, 13)  flow =  0
(11, 21)  flow =  4
(1, 10)  flow =  4
(21, 30)  flow =  3
(23, 30)  flow =  5


In [44]:
Iterate(G4)

Incremental Network: {(12, 1), (11, 10), (30, 23), (1, 11), (23, 13), (13, 22), (30, 21), (21, 11), (22, 12), (1, 12), (22, 30), (10, 1), (21, 12), (1, 13), (11, 21), (12, 21), (23, 30)}
Paths from source to sink:  {(1, 13, 22, 30)}
A best path in Gp is  (1, 13, 22, 30)  of minimum cost  4
The min capacity on this path is epsilon  5
The min cost is  4
(1, 12)  flow =  4
(10, 11)  flow =  4
(13, 23)  flow =  5
(1, 11)  flow =  0
(22, 30)  flow =  5
(21, 12)  flow =  1
(12, 22)  flow =  5
(22, 13)  flow =  0
(1, 13)  flow =  5
(11, 21)  flow =  4
(1, 10)  flow =  4
(21, 30)  flow =  3
(23, 30)  flow =  5


In [45]:
Iterate(G4)

Incremental Network: {(12, 1), (11, 10), (13, 1), (30, 23), (1, 11), (23, 13), (30, 21), (21, 11), (22, 12), (1, 12), (10, 1), (21, 12), (22, 13), (30, 22), (11, 21), (12, 21), (23, 30)}
Paths from source to sink:  set()
Maximal flow found: 13  with minimal cost  28
(1, 12)  flow =  4
(10, 11)  flow =  4
(13, 23)  flow =  5
(1, 11)  flow =  0
(22, 30)  flow =  5
(21, 12)  flow =  1
(12, 22)  flow =  5
(22, 13)  flow =  0
(1, 13)  flow =  5
(11, 21)  flow =  4
(1, 10)  flow =  4
(21, 30)  flow =  3
(23, 30)  flow =  5


In [46]:
Iterate(G4)

Incremental Network: {(12, 1), (11, 10), (13, 1), (30, 23), (1, 11), (23, 13), (30, 21), (21, 11), (22, 12), (1, 12), (10, 1), (21, 12), (22, 13), (30, 22), (11, 21), (12, 21), (23, 30)}
Paths from source to sink:  set()
Maximal flow found: 13  with minimal cost  28
(1, 12)  flow =  4
(10, 11)  flow =  4
(13, 23)  flow =  5
(1, 11)  flow =  0
(22, 30)  flow =  5
(21, 12)  flow =  1
(12, 22)  flow =  5
(22, 13)  flow =  0
(1, 13)  flow =  5
(11, 21)  flow =  4
(1, 10)  flow =  4
(21, 30)  flow =  3
(23, 30)  flow =  5


In [47]:
Iterate(G4)

Incremental Network: {(12, 1), (11, 10), (13, 1), (30, 23), (1, 11), (23, 13), (30, 21), (21, 11), (22, 12), (1, 12), (10, 1), (21, 12), (22, 13), (30, 22), (11, 21), (12, 21), (23, 30)}
Paths from source to sink:  set()
Maximal flow found: 13  with minimal cost  28
(1, 12)  flow =  4
(10, 11)  flow =  4
(13, 23)  flow =  5
(1, 11)  flow =  0
(22, 30)  flow =  5
(21, 12)  flow =  1
(12, 22)  flow =  5
(22, 13)  flow =  0
(1, 13)  flow =  5
(11, 21)  flow =  4
(1, 10)  flow =  4
(21, 30)  flow =  3
(23, 30)  flow =  5


In [48]:
Iterate(G4)

Incremental Network: {(12, 1), (11, 10), (13, 1), (30, 23), (1, 11), (23, 13), (30, 21), (21, 11), (22, 12), (1, 12), (10, 1), (21, 12), (22, 13), (30, 22), (11, 21), (12, 21), (23, 30)}
Paths from source to sink:  set()
Maximal flow found: 13  with minimal cost  28
(1, 12)  flow =  4
(10, 11)  flow =  4
(13, 23)  flow =  5
(1, 11)  flow =  0
(22, 30)  flow =  5
(21, 12)  flow =  1
(12, 22)  flow =  5
(22, 13)  flow =  0
(1, 13)  flow =  5
(11, 21)  flow =  4
(1, 10)  flow =  4
(21, 30)  flow =  3
(23, 30)  flow =  5


In [49]:
Iterate(G4)

Incremental Network: {(12, 1), (11, 10), (13, 1), (30, 23), (1, 11), (23, 13), (30, 21), (21, 11), (22, 12), (1, 12), (10, 1), (21, 12), (22, 13), (30, 22), (11, 21), (12, 21), (23, 30)}
Paths from source to sink:  set()
Maximal flow found: 13  with minimal cost  28
(1, 12)  flow =  4
(10, 11)  flow =  4
(13, 23)  flow =  5
(1, 11)  flow =  0
(22, 30)  flow =  5
(21, 12)  flow =  1
(12, 22)  flow =  5
(22, 13)  flow =  0
(1, 13)  flow =  5
(11, 21)  flow =  4
(1, 10)  flow =  4
(21, 30)  flow =  3
(23, 30)  flow =  5


In [50]:
Iterate(G4)

Incremental Network: {(12, 1), (11, 10), (13, 1), (30, 23), (1, 11), (23, 13), (30, 21), (21, 11), (22, 12), (1, 12), (10, 1), (21, 12), (22, 13), (30, 22), (11, 21), (12, 21), (23, 30)}
Paths from source to sink:  set()
Maximal flow found: 13  with minimal cost  28
(1, 12)  flow =  4
(10, 11)  flow =  4
(13, 23)  flow =  5
(1, 11)  flow =  0
(22, 30)  flow =  5
(21, 12)  flow =  1
(12, 22)  flow =  5
(22, 13)  flow =  0
(1, 13)  flow =  5
(11, 21)  flow =  4
(1, 10)  flow =  4
(21, 30)  flow =  3
(23, 30)  flow =  5


##### Iterate(G4)

In [51]:
Iterate(G4)

Incremental Network: {(12, 1), (11, 10), (13, 1), (30, 23), (1, 11), (23, 13), (30, 21), (21, 11), (22, 12), (1, 12), (10, 1), (21, 12), (22, 13), (30, 22), (11, 21), (12, 21), (23, 30)}
Paths from source to sink:  set()
Maximal flow found: 13  with minimal cost  28
(1, 12)  flow =  4
(10, 11)  flow =  4
(13, 23)  flow =  5
(1, 11)  flow =  0
(22, 30)  flow =  5
(21, 12)  flow =  1
(12, 22)  flow =  5
(22, 13)  flow =  0
(1, 13)  flow =  5
(11, 21)  flow =  4
(1, 10)  flow =  4
(21, 30)  flow =  3
(23, 30)  flow =  5


In [52]:
Iterate(G4)

Incremental Network: {(12, 1), (11, 10), (13, 1), (30, 23), (1, 11), (23, 13), (30, 21), (21, 11), (22, 12), (1, 12), (10, 1), (21, 12), (22, 13), (30, 22), (11, 21), (12, 21), (23, 30)}
Paths from source to sink:  set()
Maximal flow found: 13  with minimal cost  28
(1, 12)  flow =  4
(10, 11)  flow =  4
(13, 23)  flow =  5
(1, 11)  flow =  0
(22, 30)  flow =  5
(21, 12)  flow =  1
(12, 22)  flow =  5
(22, 13)  flow =  0
(1, 13)  flow =  5
(11, 21)  flow =  4
(1, 10)  flow =  4
(21, 30)  flow =  3
(23, 30)  flow =  5


## Q B 500 Kg

In [53]:
G5 = {(1, 11), (1, 12), (1, 13), (1, 14),
      (1, 15), (1, 16), (1, 17), (1, 18),
      (1, 19), (1, 20), (1, 21), (1, 22),
      (11, 31), (12, 32), (13, 33), (14, 34),
      (15, 35), (16, 36), (17, 37), (18, 38),
      (19, 39), (20, 40), (21, 41), (22, 42),
      (31, 12), (32, 13), (33, 14), (34, 15),
      (35, 16), (36, 17), (37, 18), (38, 19),
      (39, 20), (40, 21), (41, 22),
      (31, 50), (32, 50), (33, 50), (34, 50),
      (35, 50), (36, 50), (37, 50), (38, 50),
      (39, 50), (40, 50), (41, 50), (42, 50),
      
      (1, 5), (5,11),
      (42, 45), (45,50)} 

In [54]:
Initialise20(G5)

All values of phi and l initialised to zero and c to 20


In [55]:
c[31][50] = 9; c[32][50] = 6; c[33][50] = 6;
c[34][50] = 7; c[35][50] = 11; c[36][50] = 14;
c[37][50] = 16; c[38][50] = 18; c[39][50] = 15;
c[40][50] = 10; c[41][50] = 7; c[42][50] = 6;

c[1][5] = 5; c[5][11] = 5;
c[42][45] = 5; c[45][50] = 5;

In [56]:
l[1][11] = 18; l[1][12] = 17; l[1][13] = 17;
l[1][14] = 15; l[1][15] = 12; l[1][16] = 8;
l[1][17] = 7; l[1][18] = 6; l[1][19] = 9;
l[1][20] = 12; l[1][21] = 14; l[1][22] = 17;

l[11][31] = 1; l[12][32] = 1; l[13][33] = 2;
l[14][34] = 2; l[15][35] = 3; l[16][36] = 5;
l[17][37] = 6; l[18][38] = 6; l[19][39] = 5;
l[20][40] = 3; l[21][41] = 2; l[22][42] = 1;


In [57]:
Flows(G5)

(42, 45)  has flow  0
(13, 33)  has flow  0
(18, 38)  has flow  0
(37, 18)  has flow  0
(38, 50)  has flow  0
(40, 50)  has flow  0
(33, 14)  has flow  0
(14, 34)  has flow  0
(31, 50)  has flow  0
(1, 12)  has flow  0
(1, 18)  has flow  0
(36, 50)  has flow  0
(1, 15)  has flow  0
(1, 21)  has flow  0
(38, 19)  has flow  0
(19, 39)  has flow  0
(39, 20)  has flow  0
(42, 50)  has flow  0
(15, 35)  has flow  0
(33, 50)  has flow  0
(34, 15)  has flow  0
(35, 50)  has flow  0
(20, 40)  has flow  0
(39, 50)  has flow  0
(1, 11)  has flow  0
(1, 5)  has flow  0
(1, 14)  has flow  0
(1, 20)  has flow  0
(35, 16)  has flow  0
(1, 17)  has flow  0
(11, 31)  has flow  0
(37, 50)  has flow  0
(31, 12)  has flow  0
(40, 21)  has flow  0
(45, 50)  has flow  0
(41, 50)  has flow  0
(16, 36)  has flow  0
(32, 50)  has flow  0
(5, 11)  has flow  0
(36, 17)  has flow  0
(12, 32)  has flow  0
(21, 41)  has flow  0
(32, 13)  has flow  0
(22, 42)  has flow  0
(1, 13)  has flow  0
(17, 37)  has flow  0


In [58]:
Iterate(G5)

Incremental Network: {(42, 45), (13, 33), (18, 38), (37, 18), (38, 50), (34, 50), (40, 50), (33, 14), (14, 34), (31, 50), (1, 12), (1, 18), (1, 15), (1, 21), (38, 19), (19, 39), (39, 20), (42, 50), (15, 35), (33, 50), (34, 15), (35, 50), (20, 40), (39, 50), (1, 11), (1, 5), (1, 14), (1, 20), (35, 16), (1, 17), (11, 31), (37, 50), (31, 12), (40, 21), (41, 50), (16, 36), (32, 50), (5, 11), (36, 17), (1, 22), (12, 32), (21, 41), (32, 13), (22, 42), (1, 13), (17, 37), (1, 16), (41, 22), (45, 50), (1, 19), (36, 50)}
Paths from source to sink:  {(1, 16, 36, 17, 37, 50), (1, 5, 11, 31, 12, 32, 13, 33, 14, 34, 15, 35, 16, 36, 17, 37, 18, 38, 19, 39, 20, 40, 21, 41, 22, 42, 45, 50), (1, 11, 31, 12, 32, 13, 33, 14, 34, 50), (1, 17, 37, 18, 38, 19, 39, 20, 40, 50), (1, 11, 31, 12, 32, 13, 33, 14, 34, 15, 35, 16, 36, 17, 37, 18, 38, 19, 39, 20, 40, 21, 41, 22, 42, 50), (1, 18, 38, 19, 39, 20, 40, 21, 41, 22, 42, 50), (1, 13, 33, 14, 34, 50), (1, 21, 41, 22, 42, 50), (1, 5, 11, 31, 12, 32, 13, 33, 

In [59]:
Iterate(G5)

Incremental Network: {(31, 11), (42, 45), (13, 33), (1, 16), (5, 1), (18, 38), (37, 18), (38, 50), (11, 5), (34, 50), (40, 50), (33, 14), (14, 34), (31, 50), (1, 12), (1, 18), (1, 15), (1, 21), (38, 19), (19, 39), (39, 20), (42, 50), (15, 35), (33, 50), (34, 15), (35, 50), (20, 40), (39, 50), (1, 11), (1, 14), (1, 20), (35, 16), (1, 17), (11, 31), (37, 50), (31, 12), (40, 21), (41, 50), (16, 36), (32, 50), (36, 17), (1, 22), (12, 32), (21, 41), (32, 13), (22, 42), (1, 13), (50, 31), (17, 37), (41, 22), (45, 50), (1, 19), (36, 50)}
Paths from source to sink:  {(1, 16, 36, 17, 37, 50), (1, 11, 31, 12, 32, 13, 33, 14, 34, 50), (1, 17, 37, 18, 38, 19, 39, 20, 40, 50), (1, 11, 31, 12, 32, 13, 33, 14, 34, 15, 35, 16, 36, 17, 37, 18, 38, 19, 39, 20, 40, 21, 41, 22, 42, 50), (1, 18, 38, 19, 39, 20, 40, 21, 41, 22, 42, 50), (1, 13, 33, 14, 34, 50), (1, 21, 41, 22, 42, 50), (1, 12, 32, 13, 33, 14, 34, 15, 35, 16, 36, 17, 37, 18, 38, 50), (1, 14, 34, 15, 35, 16, 36, 17, 37, 18, 38, 19, 39, 20, 40

In [60]:
Iterate(G5)

Incremental Network: {(31, 11), (42, 45), (13, 33), (1, 16), (5, 1), (18, 38), (37, 18), (11, 5), (34, 50), (40, 50), (33, 14), (14, 34), (31, 50), (1, 12), (1, 18), (18, 1), (1, 15), (1, 21), (38, 19), (19, 39), (39, 20), (42, 50), (15, 35), (33, 50), (34, 15), (35, 50), (20, 40), (39, 50), (1, 11), (1, 14), (1, 20), (35, 16), (50, 38), (38, 18), (1, 17), (11, 31), (37, 50), (31, 12), (40, 21), (41, 50), (16, 36), (32, 50), (36, 17), (1, 22), (12, 32), (21, 41), (32, 13), (22, 42), (1, 13), (50, 31), (17, 37), (41, 22), (45, 50), (1, 19), (36, 50)}
Paths from source to sink:  {(1, 16, 36, 17, 37, 50), (1, 11, 31, 12, 32, 13, 33, 14, 34, 50), (1, 17, 37, 18, 38, 19, 39, 20, 40, 50), (1, 11, 31, 12, 32, 13, 33, 14, 34, 15, 35, 16, 36, 17, 37, 18, 38, 19, 39, 20, 40, 21, 41, 22, 42, 50), (1, 18, 38, 19, 39, 20, 40, 21, 41, 22, 42, 50), (1, 13, 33, 14, 34, 50), (1, 21, 41, 22, 42, 50), (1, 14, 34, 15, 35, 16, 36, 17, 37, 18, 38, 19, 39, 20, 40, 21, 41, 22, 42, 50), (1, 18, 38, 19, 39, 50)

In [61]:
Iterate(G5)

Incremental Network: {(31, 11), (42, 45), (13, 33), (1, 16), (5, 1), (36, 16), (18, 38), (37, 18), (11, 5), (34, 50), (40, 50), (33, 14), (14, 34), (31, 50), (1, 12), (1, 18), (16, 1), (18, 1), (50, 36), (1, 15), (1, 21), (38, 19), (19, 39), (39, 20), (42, 50), (15, 35), (33, 50), (34, 15), (35, 50), (20, 40), (39, 50), (1, 11), (1, 14), (1, 20), (35, 16), (50, 38), (38, 18), (1, 17), (11, 31), (37, 50), (31, 12), (40, 21), (41, 50), (16, 36), (32, 50), (36, 17), (12, 32), (21, 41), (32, 13), (22, 42), (1, 13), (50, 31), (17, 37), (41, 22), (45, 50), (1, 19), (1, 22)}
Paths from source to sink:  {(1, 16, 36, 17, 37, 50), (1, 11, 31, 12, 32, 13, 33, 14, 34, 50), (1, 17, 37, 18, 38, 19, 39, 20, 40, 50), (1, 11, 31, 12, 32, 13, 33, 14, 34, 15, 35, 16, 36, 17, 37, 18, 38, 19, 39, 20, 40, 21, 41, 22, 42, 50), (1, 18, 38, 19, 39, 20, 40, 21, 41, 22, 42, 50), (1, 13, 33, 14, 34, 50), (1, 21, 41, 22, 42, 50), (1, 14, 34, 15, 35, 16, 36, 17, 37, 18, 38, 19, 39, 20, 40, 21, 41, 22, 42, 50), (1, 

In [62]:
Iterate(G5)

Incremental Network: {(31, 11), (42, 45), (13, 33), (1, 16), (5, 1), (36, 16), (18, 38), (37, 18), (11, 5), (34, 50), (40, 50), (33, 14), (14, 34), (31, 50), (1, 12), (1, 18), (16, 1), (18, 1), (50, 36), (1, 15), (1, 21), (38, 19), (19, 39), (39, 20), (42, 50), (15, 35), (33, 50), (34, 15), (35, 50), (37, 17), (20, 40), (39, 50), (1, 11), (1, 14), (1, 20), (35, 16), (50, 38), (38, 18), (1, 17), (11, 31), (31, 12), (40, 21), (41, 50), (16, 36), (17, 1), (32, 50), (36, 17), (12, 32), (21, 41), (32, 13), (22, 42), (1, 13), (50, 31), (50, 37), (17, 37), (41, 22), (45, 50), (1, 19), (1, 22)}
Paths from source to sink:  {(1, 11, 31, 12, 32, 13, 33, 14, 34, 50), (1, 17, 37, 18, 38, 19, 39, 20, 40, 50), (1, 11, 31, 12, 32, 13, 33, 14, 34, 15, 35, 16, 36, 17, 37, 18, 38, 19, 39, 20, 40, 21, 41, 22, 42, 50), (1, 18, 38, 19, 39, 20, 40, 21, 41, 22, 42, 50), (1, 13, 33, 14, 34, 50), (1, 21, 41, 22, 42, 50), (1, 14, 34, 15, 35, 16, 36, 17, 37, 18, 38, 19, 39, 20, 40, 21, 41, 22, 42, 50), (1, 18, 38

In [63]:
Iterate(G5)

Incremental Network: {(31, 11), (42, 45), (13, 33), (1, 16), (5, 1), (36, 16), (18, 38), (37, 18), (11, 5), (34, 50), (40, 50), (33, 14), (14, 34), (31, 50), (1, 12), (1, 18), (16, 1), (18, 1), (50, 36), (1, 15), (1, 21), (50, 39), (38, 19), (19, 39), (39, 20), (42, 50), (15, 35), (33, 50), (34, 15), (35, 50), (37, 17), (20, 40), (1, 11), (1, 14), (1, 20), (35, 16), (50, 38), (38, 18), (1, 17), (11, 31), (31, 12), (39, 19), (40, 21), (41, 50), (16, 36), (17, 1), (32, 50), (19, 1), (36, 17), (12, 32), (21, 41), (32, 13), (22, 42), (1, 13), (50, 31), (50, 37), (17, 37), (41, 22), (45, 50), (1, 19), (1, 22)}
Paths from source to sink:  {(1, 14, 34, 50), (1, 11, 31, 12, 32, 13, 33, 14, 34, 50), (1, 13, 33, 14, 34, 15, 35, 16, 36, 17, 37, 18, 38, 19, 39, 20, 40, 21, 41, 50), (1, 17, 37, 18, 38, 19, 39, 20, 40, 50), (1, 12, 32, 13, 33, 14, 34, 15, 35, 16, 36, 17, 37, 18, 38, 19, 39, 20, 40, 50), (1, 11, 31, 12, 32, 13, 33, 14, 34, 15, 35, 16, 36, 17, 37, 18, 38, 19, 39, 20, 40, 21, 41, 22, 4

In [64]:
Iterate(G5)

Incremental Network: {(31, 11), (42, 45), (13, 33), (40, 20), (1, 16), (5, 1), (36, 16), (18, 38), (37, 18), (11, 5), (34, 50), (33, 14), (14, 34), (31, 50), (1, 12), (1, 18), (16, 1), (18, 1), (50, 36), (1, 15), (1, 21), (50, 39), (38, 19), (20, 1), (19, 39), (39, 20), (42, 50), (15, 35), (33, 50), (34, 15), (35, 50), (37, 17), (20, 40), (1, 11), (1, 14), (1, 20), (35, 16), (50, 38), (38, 18), (1, 17), (11, 31), (31, 12), (39, 19), (40, 21), (41, 50), (16, 36), (17, 1), (32, 50), (1, 19), (19, 1), (36, 17), (12, 32), (21, 41), (32, 13), (22, 42), (1, 13), (50, 31), (50, 37), (17, 37), (41, 22), (45, 50), (50, 40), (1, 22)}
Paths from source to sink:  {(1, 14, 34, 50), (1, 11, 31, 12, 32, 13, 33, 14, 34, 50), (1, 13, 33, 14, 34, 15, 35, 16, 36, 17, 37, 18, 38, 19, 39, 20, 40, 21, 41, 50), (1, 11, 31, 12, 32, 13, 33, 14, 34, 15, 35, 16, 36, 17, 37, 18, 38, 19, 39, 20, 40, 21, 41, 22, 42, 50), (1, 18, 38, 19, 39, 20, 40, 21, 41, 22, 42, 50), (1, 13, 33, 14, 34, 50), (1, 21, 41, 22, 42, 5

In [65]:
Iterate(G5)

Incremental Network: {(13, 33), (5, 1), (36, 16), (11, 5), (33, 14), (1, 18), (18, 1), (50, 36), (1, 15), (19, 39), (39, 20), (42, 50), (37, 17), (20, 40), (1, 11), (1, 20), (50, 38), (38, 18), (1, 17), (11, 31), (40, 21), (39, 19), (45, 50), (41, 50), (19, 1), (36, 17), (21, 41), (1, 13), (50, 31), (1, 19), (50, 40), (31, 11), (42, 45), (40, 20), (18, 38), (37, 18), (14, 34), (31, 50), (1, 12), (16, 1), (1, 21), (50, 39), (38, 19), (20, 1), (50, 37), (15, 35), (33, 50), (34, 15), (15, 1), (50, 35), (1, 14), (35, 16), (35, 15), (31, 12), (16, 36), (17, 1), (32, 50), (12, 32), (32, 13), (22, 42), (17, 37), (1, 16), (41, 22), (34, 50), (1, 22)}
Paths from source to sink:  {(1, 14, 34, 50), (1, 11, 31, 12, 32, 13, 33, 14, 34, 50), (1, 13, 33, 14, 34, 15, 35, 16, 36, 17, 37, 18, 38, 19, 39, 20, 40, 21, 41, 50), (1, 11, 31, 12, 32, 13, 33, 14, 34, 15, 35, 16, 36, 17, 37, 18, 38, 19, 39, 20, 40, 21, 41, 22, 42, 50), (1, 18, 38, 19, 39, 20, 40, 21, 41, 22, 42, 50), (1, 13, 33, 14, 34, 50), (1

In [66]:
Iterate(G5)

Incremental Network: {(13, 33), (5, 1), (36, 16), (11, 5), (33, 14), (1, 18), (18, 1), (50, 36), (1, 15), (19, 39), (39, 20), (42, 50), (37, 17), (20, 40), (1, 11), (1, 20), (50, 38), (38, 18), (1, 17), (11, 31), (40, 21), (39, 19), (45, 50), (19, 1), (36, 17), (21, 41), (1, 13), (50, 31), (50, 40), (1, 19), (21, 1), (31, 11), (42, 45), (40, 20), (18, 38), (37, 18), (41, 21), (14, 34), (31, 50), (1, 12), (16, 1), (1, 21), (50, 39), (38, 19), (20, 1), (50, 37), (15, 35), (33, 50), (34, 15), (15, 1), (50, 35), (1, 14), (35, 16), (35, 15), (50, 41), (31, 12), (16, 36), (17, 1), (32, 50), (12, 32), (32, 13), (22, 42), (17, 37), (1, 16), (41, 22), (34, 50), (1, 22)}
Paths from source to sink:  {(1, 14, 34, 50), (1, 11, 31, 12, 32, 13, 33, 14, 34, 50), (1, 11, 31, 12, 32, 13, 33, 14, 34, 15, 35, 16, 36, 17, 37, 18, 38, 19, 39, 20, 40, 21, 41, 22, 42, 50), (1, 18, 38, 19, 39, 20, 40, 21, 41, 22, 42, 50), (1, 13, 33, 14, 34, 50), (1, 21, 41, 22, 42, 50), (1, 14, 34, 15, 35, 16, 36, 17, 37, 18,

In [67]:
Iterate(G5)

Incremental Network: {(13, 33), (5, 1), (36, 16), (11, 5), (33, 14), (1, 18), (18, 1), (50, 36), (1, 15), (19, 39), (39, 20), (42, 50), (37, 17), (20, 40), (1, 11), (1, 20), (50, 38), (38, 18), (1, 17), (11, 31), (40, 21), (39, 19), (34, 14), (19, 1), (36, 17), (21, 41), (1, 13), (50, 31), (50, 40), (1, 19), (21, 1), (31, 11), (42, 45), (40, 20), (14, 1), (18, 38), (37, 18), (41, 21), (14, 34), (31, 50), (1, 12), (16, 1), (1, 21), (50, 39), (38, 19), (20, 1), (50, 37), (50, 34), (15, 35), (33, 50), (34, 15), (15, 1), (50, 35), (1, 14), (35, 16), (35, 15), (50, 41), (31, 12), (16, 36), (17, 1), (32, 50), (12, 32), (32, 13), (22, 42), (17, 37), (1, 16), (41, 22), (45, 50), (1, 22)}
Paths from source to sink:  {(1, 15, 35, 16, 36, 17, 37, 18, 38, 19, 39, 20, 40, 21, 41, 22, 42, 50), (1, 11, 31, 12, 32, 13, 33, 50), (1, 17, 37, 18, 38, 19, 39, 20, 40, 21, 41, 22, 42, 45, 50), (1, 16, 36, 17, 37, 18, 38, 19, 39, 20, 40, 21, 41, 22, 42, 45, 50), (1, 11, 31, 12, 32, 13, 33, 14, 34, 15, 35, 16

In [68]:
Iterate(G5)

Incremental Network: {(13, 33), (5, 1), (36, 16), (11, 5), (33, 14), (22, 41), (1, 18), (18, 1), (50, 36), (1, 15), (19, 39), (39, 20), (37, 17), (20, 40), (1, 11), (1, 20), (50, 38), (38, 18), (1, 17), (11, 31), (40, 21), (39, 19), (34, 14), (19, 1), (36, 17), (21, 41), (1, 13), (50, 31), (50, 40), (1, 19), (21, 1), (31, 11), (42, 45), (40, 20), (14, 1), (18, 38), (37, 18), (41, 21), (14, 34), (31, 50), (1, 12), (16, 1), (1, 21), (50, 42), (50, 39), (38, 19), (20, 1), (50, 37), (50, 34), (15, 35), (33, 50), (34, 15), (42, 22), (15, 1), (50, 35), (1, 14), (35, 16), (35, 15), (50, 41), (31, 12), (16, 36), (17, 1), (32, 50), (12, 32), (32, 13), (22, 42), (17, 37), (1, 16), (41, 22), (45, 50), (1, 22)}
Paths from source to sink:  {(1, 11, 31, 12, 32, 13, 33, 50), (1, 17, 37, 18, 38, 19, 39, 20, 40, 21, 41, 22, 42, 45, 50), (1, 16, 36, 17, 37, 18, 38, 19, 39, 20, 40, 21, 41, 22, 42, 45, 50), (1, 14, 34, 15, 35, 16, 36, 17, 37, 18, 38, 19, 39, 20, 40, 21, 41, 22, 42, 45, 50), (1, 11, 31, 50

In [69]:
Iterate(G5)

Incremental Network: {(13, 33), (5, 1), (36, 16), (11, 5), (33, 14), (22, 41), (1, 18), (18, 1), (50, 36), (1, 15), (50, 45), (19, 39), (39, 20), (37, 17), (20, 40), (1, 11), (1, 20), (50, 38), (38, 18), (1, 17), (11, 31), (40, 21), (39, 19), (34, 14), (19, 1), (36, 17), (21, 41), (1, 13), (50, 31), (50, 40), (1, 19), (21, 1), (31, 11), (40, 20), (14, 1), (18, 38), (37, 18), (41, 21), (14, 34), (31, 50), (1, 12), (16, 1), (1, 21), (50, 42), (50, 39), (38, 19), (20, 1), (50, 37), (50, 34), (15, 35), (33, 50), (34, 15), (45, 42), (42, 22), (15, 1), (50, 35), (1, 14), (35, 16), (50, 41), (31, 12), (16, 36), (17, 1), (32, 50), (12, 32), (32, 13), (22, 42), (17, 37), (1, 16), (41, 22), (35, 15), (1, 22)}
Paths from source to sink:  {(1, 13, 33, 50), (1, 11, 31, 12, 32, 13, 33, 50), (1, 11, 31, 50), (1, 11, 31, 12, 32, 50), (1, 12, 32, 13, 33, 50), (1, 12, 32, 50)}
A best path in Gp is  (1, 12, 32, 50)  of minimum cost  18
The min capacity on this path is epsilon  6
The min cost is  18
(42, 

In [70]:
Iterate(G5)

Incremental Network: {(12, 1), (13, 33), (1, 16), (5, 1), (36, 16), (11, 5), (33, 14), (32, 12), (22, 41), (1, 18), (18, 1), (50, 36), (1, 15), (50, 45), (19, 39), (39, 20), (37, 17), (20, 40), (1, 11), (50, 32), (1, 20), (50, 38), (38, 18), (1, 17), (11, 31), (40, 21), (39, 19), (34, 14), (19, 1), (36, 17), (21, 41), (1, 13), (50, 31), (50, 40), (1, 19), (21, 1), (31, 11), (40, 20), (14, 1), (18, 38), (37, 18), (41, 21), (14, 34), (31, 50), (1, 12), (16, 1), (1, 21), (50, 42), (50, 39), (38, 19), (20, 1), (50, 34), (15, 35), (33, 50), (34, 15), (45, 42), (42, 22), (15, 1), (50, 35), (1, 14), (35, 16), (50, 41), (31, 12), (16, 36), (17, 1), (12, 32), (32, 13), (22, 42), (17, 37), (50, 37), (41, 22), (35, 15), (1, 22)}
Paths from source to sink:  {(1, 12, 32, 13, 33, 50), (1, 13, 33, 50), (1, 11, 31, 12, 32, 13, 33, 50), (1, 11, 31, 50)}
A best path in Gp is  (1, 13, 33, 50)  of minimum cost  19
The min capacity on this path is epsilon  6
The min cost is  19
(42, 45)  flow =  5
(13, 33)

In [71]:
Iterate(G5)

Incremental Network: {(12, 1), (13, 33), (1, 16), (5, 1), (36, 16), (11, 5), (33, 14), (32, 12), (22, 41), (1, 18), (18, 1), (50, 36), (1, 15), (50, 45), (19, 39), (39, 20), (37, 17), (20, 40), (13, 1), (1, 11), (50, 32), (33, 13), (1, 20), (50, 38), (38, 18), (1, 17), (11, 31), (40, 21), (39, 19), (34, 14), (19, 1), (36, 17), (21, 41), (1, 13), (50, 31), (50, 40), (1, 19), (21, 1), (31, 11), (40, 20), (14, 1), (18, 38), (37, 18), (41, 21), (14, 34), (31, 50), (1, 12), (50, 33), (16, 1), (1, 21), (50, 42), (50, 39), (38, 19), (20, 1), (50, 34), (15, 35), (34, 15), (45, 42), (42, 22), (15, 1), (50, 35), (1, 14), (35, 16), (50, 41), (31, 12), (16, 36), (17, 1), (12, 32), (32, 13), (22, 42), (17, 37), (50, 37), (41, 22), (35, 15), (1, 22)}
Paths from source to sink:  {(1, 11, 31, 50)}
A best path in Gp is  (1, 11, 31, 50)  of minimum cost  19
The min capacity on this path is epsilon  4
The min cost is  19
(42, 45)  flow =  5
(13, 33)  flow =  6
(18, 38)  flow =  18
(37, 18)  flow =  0
(38

In [72]:
Iterate(G5)

Incremental Network: {(12, 1), (13, 33), (1, 16), (5, 1), (36, 16), (11, 5), (33, 14), (32, 12), (22, 41), (1, 18), (18, 1), (50, 36), (1, 15), (50, 45), (19, 39), (39, 20), (37, 17), (20, 40), (13, 1), (1, 11), (50, 32), (33, 13), (1, 20), (50, 38), (38, 18), (1, 17), (11, 31), (40, 21), (39, 19), (34, 14), (19, 1), (36, 17), (21, 41), (1, 13), (50, 31), (50, 40), (1, 19), (21, 1), (31, 11), (40, 20), (14, 1), (18, 38), (37, 18), (41, 21), (14, 34), (16, 1), (1, 12), (50, 33), (1, 21), (50, 42), (50, 39), (38, 19), (20, 1), (50, 34), (15, 35), (34, 15), (11, 1), (45, 42), (42, 22), (15, 1), (50, 35), (1, 14), (35, 16), (50, 41), (31, 12), (16, 36), (17, 1), (12, 32), (32, 13), (22, 42), (17, 37), (50, 37), (41, 22), (35, 15), (1, 22)}
Paths from source to sink:  set()
Maximal flow found: 130  with minimal cost  1852
(42, 45)  flow =  5
(13, 33)  flow =  6
(18, 38)  flow =  18
(37, 18)  flow =  0
(38, 50)  flow =  18
(40, 50)  flow =  10
(33, 14)  flow =  0
(14, 34)  flow =  7
(31, 50)